In [ ]:
# Import packages

%pip install fastf1 matplotlib-label-lines --quiet
import matplotlib.pyplot as plt
import matplotlib.style as style
# import numpy as np

import matplotlib.pyplot as plt
import matplotlib.ticker as tick
import matplotlib.lines as mlines
from matplotlib.colors import to_rgba

import fastf1
import fastf1.plotting
from fastf1.ergast import Ergast
import pandas as pd
import numpy as np

from labellines import labelLines
import urllib.request, json
import datetime
import seaborn as sns

# Load FastF1's dark color scheme
fastf1.plotting.setup_mpl(color_scheme='fastf1')

# load widget and interactive
from ipywidgets import interact, interactive, fixed, interact_manual
import ipywidgets as widgets

# global variables
event_path = None
lapSeries = None
driverList = None
sessionInfo = None
session_idx = None
session_list = None

year = 2025
with urllib.request.urlopen(f"https://livetiming.formula1.com/static/{year}/Index.json") as url:
    seasonInfo = json.load(url)

sessions = [ (f"{meeting['Name']} - {session['Name']}", session.get('Path')) for meeting in seasonInfo['Meetings'] for session in meeting['Sessions'] if session.get('Path') ]
sessions.reverse()

session_number_mapping = {
    "Practice 1": 1,
    "Practice 2": 2,
    "Practice 3": 3,
    "Day 1": 1,
    "Day 2": 2,
    "Day 3": 3,
    "Sprint Qualifying": 2,
    "Sprint": 3,
    "Qualifying": 4,
    "Race": 5

}

@interact(event_path=sessions)
def print_event_path(event_path):
    global sessionInfo
    global driverList
    global session_idx
    global lapSeries
    global session_list
    with urllib.request.urlopen(f"https://livetiming.formula1.com/static/{event_path}SessionInfo.json") as url:
        sessionInfo = json.load(url)
    with urllib.request.urlopen(f"https://livetiming.formula1.com/static/{event_path}DriverList.json") as url:
        driverList = json.load(url)
    with urllib.request.urlopen(f"https://livetiming.formula1.com/static/{event_path}LapSeries.json") as url:
        lapSeries = json.load(url)
    session_idx ={
        'year': datetime.datetime.strptime(sessionInfo['StartDate'], '%Y-%m-%dT%H:%M:%S').year,
        'event': sessionInfo['Meeting']['Name'],
        'session': session_number_mapping[sessionInfo['Name']] - int( "Complete" != sessionInfo['ArchiveStatus']['Status'] )
    }
    session_list = [
        fastf1.get_session(session_idx['year'], session_idx['event'], i)
        for i in range(1, 1 + session_idx['session'])
    ]
    print(event_path)

## Broken Line graph showing position change in Race

In [ ]:

fig, ax = plt.subplots(figsize=(12.0, 6.0))
fig.suptitle('POsition change')

driver_style ={key: {'color': f"#{info['TeamColour']}", 'linestyle': ['solid', 'dashed'][idx%2]} for idx, (key, info) in enumerate(sorted( driverList.items(), key=lambda item: item[1]['TeamColour'] ))}

xvals=[]
for drv, info in sorted(driverList.items(), key=lambda item: item[1]['TeamName']):
    abb = info["Tla"]
    style = driver_style[drv]
    lap_no = [ i for i in range(len(lapSeries[drv]['LapPosition'])) ]
    lap_pos = [ int(i) for i in lapSeries[drv]['LapPosition'] ]
    xvals.append(max(lap_no))
    ax.plot(lap_no, lap_pos,
            label=info['Tla'], **style)

ax.set_ylim([len(driver_style.items())+1, 0])
ax.set_yticks([1, 5, 10, 15, 20])
ax.set_xlabel('LAP')
ax.set_ylabel('POS')

fig.tight_layout()
labelLines(ax.get_lines(), align=False, xvals=xvals)
ax.grid(axis = 'x', linestyle = '--')
fig.show()

## Boxplot showing pace

In [ ]:
def pace_plot(plot_type, season_idx, event_idx, session_idx, driverList):
    """
    Generates a pace analysis plot for drivers or teams.

    This function fetches lap data from completed sessions of a given F1 event,
    processes it, and creates a box plot overlaid with a swarm plot to visualize
    lap time distributions. The plot can be grouped by driver or by team.

    Args:
        plot_type (str): The type of plot to generate, either 'driver' or 'team'.
        season_idx (int): The year of the season.
        event_idx (str or int): The name or round number of the event.
        session_idx (int): The number of the last completed session to include data from.
        driverList (dict): A dictionary of live driver data from Redis, used for
                           ordering and team color information.

    Returns:
        matplotlib.figure.Figure or None: The generated plot figure, or None if
                                          no data is available.
    """
    # --- Historical Data Loading (FastF1) ---
    # Create a list of all completed FastF1 session objects for the current event.
    session_list = [
        fastf1.get_session(season_idx, event_idx, i)
        for i in range(1, 1 + session_idx)
    ]

    # Load the data for each session. This can be time-consuming.
    # We disable telemetry loading as we only need lap data.
    for session in session_list:
        session.load(telemetry=False, weather=True, messages=True)

    if len(session_list) == 0:
        return None

    # --- Data Aggregation & Cleaning ---
    # Get a list of driver numbers, sorted by their current position on the timing screen.
    # This ensures the plot's x-axis is ordered by the current race/quali standings.
    drivers = [
        key
        for key, _ in sorted(
            driverList.items(), key=lambda item: int(item[1]["Line"])
        )
    ]

    # Create a color palette for tyre compounds.
    tire_palette = msgStyle["compoundRGB"]
    compounds = list(tire_palette.keys())

    # For each session, aggregate all valid laps for the specified drivers.
    # A chain of FastF1 filters is applied to ensure data quality and relevance:
    # - pick_drivers():     Selects laps only for the drivers currently on track.
    # - pick_wo_box():      Excludes in-laps and out-laps (laps entering/leaving pits).
    # - pick_not_deleted(): Excludes laps invalidated by race control (e.g., for track limits).
    # - pick_accurate():    Excludes laps with known timing inaccuracies.
    # - pick_compounds():   Includes only laps set on standard race compounds (W, I, S, M, H).
    # - pick_track_status("1"): Includes only laps set under green flag conditions.
    driver_laps_per_session = [
        session.laps.pick_drivers(drivers)
        .pick_wo_box()
        .pick_not_deleted()
        .pick_accurate()
        .pick_compounds(compounds)
        .pick_track_status("1")
        for session in session_list
    ]
    for idx, session_laps in enumerate(driver_laps_per_session):
        session_laps["Session_Type"] = session_list[idx].session_info['Type']
        session_laps['Session_Number'] = idx
    # Combine the laps from all sessions into a single pandas DataFrame.
    driver_laps = pd.concat(driver_laps_per_session)
    driver_laps = driver_laps.reset_index()

    # --- Plotting Setup ---
    # Determine the order of drivers on the x-axis based on their current timing screen position.
    driver_order = [driverList[i]["Tla"] for i in drivers]
    team_order = list(dict.fromkeys([driverList[i]["TeamName"] for i in drivers]))
    # Create a color palette mapping each driver's TLA to their team color.
    driver_palette = {
        value["Tla"]: f"#{value['TeamColour']}"
        for _, value in driverList.items()
    }
    driver_palette_wet = { key: to_rgba(val, alpha=0.5) for key, val in driver_palette.items() }
    session_type_marker = {
        "Race": "o",
        "Qualifying": 7,
        "Practice": "s",
    }

    # Initialize the matplotlib figure and axes.
    fig, ax = plt.subplots(figsize=(21, 9))
    fig.suptitle(f"{season_idx} {event_idx} {plot_type} pace".title())
    ax.set_xlabel("Driver")
    ax.set_ylabel("Lap Time")
    # ax.grid(axis="y", linestyle="--")
    ax.set_xticks(np.arange(-0.5, 30, 1), minor=True)
    ax.grid(which="major",axis = 'y', linestyle = '--')
    ax.grid(which="minor",axis = 'x', linestyle = '--')
    ax.grid(which="minor",axis = 'y', linestyle = ':', linewidth=0.5)
    ax.yaxis.set_minor_locator(tick.AutoMinorLocator())
    time_formatter = tick.FuncFormatter( lambda x, y: f"{int(x//60)}:{int(x%60):02}")
    ax.yaxis.set_major_formatter(time_formatter)
    ax.legend(handles=[
        mlines.Line2D([], [], marker=marker, label=label, linestyle='None', markersize=10, color='white')
        for label, marker in session_type_marker.items()
    ])
    fig.tight_layout()

    # Convert the 'LapTime' (a timedelta object) to total seconds for plotting on a numeric axis.
    driver_laps["LapTime(s)"] = driver_laps["LapTime"].dt.total_seconds()
    threshold = min(
        [driver_laps["LapTime(s)"].min() * 1.2, driver_laps["LapTime(s)"].min() + 20.0]
    )
    driver_laps = driver_laps[
        (driver_laps["LapTime(s)"] <= threshold)
        | (driver_laps["Session_Type"] == "Race")
    ]

    used_compounds = sorted(
        driver_laps["Compound"].unique(),
        key=lambda x: compounds.index(x)
    )

    if plot_type == 'driver':
        # 1. Create the box plot to show the distribution of lap times for each driver.
        #    This gives a good overview of each driver's pace consistency.
        # Dry Tires
        sns.boxplot(
            data=driver_laps[ driver_laps['Compound'].isin(['SOFT', 'MEDIUM', 'HARD']) ],
            x="Driver",
            y="LapTime(s)",
            hue="Driver",
            order=driver_order,
            palette=driver_palette,
            fill=False,
            showfliers=False,
            legend=False,
            saturation=1,
        )

        # Wet Tires
        sns.boxplot(
            data=driver_laps[ driver_laps['Compound'].isin(['WET', 'INTERMEDIATE']) ],
            x="Driver",
            y="LapTime(s)",
            hue="Driver",
            order=driver_order,
            palette=driver_palette_wet,
            fill=False,
            showfliers=False,
            legend=False,
            # saturation=0.1,
        )

        # 2. Overlay a swarm plot to show each individual valid lap.
        #    Each point is colored by the tyre compound used for that lap, providing
        #    deeper insight into the pace on different compounds.
        for session_no in range(session_idx):
            session_type = session_list[session_no].session_info['Type']
            marker = session_type_marker[session_type]
            tire_palette_adj = { key: to_rgba(val, alpha=(session_no + 1.)/session_idx) for key, val in tire_palette.items() }
            sns.swarmplot(
                data=driver_laps[ driver_laps['Session_Number'] == session_no ],
                x="Driver",
                y="LapTime(s)",
                order=driver_order,
                hue="Compound",
                palette=tire_palette_adj,
                hue_order=used_compounds,
                linewidth=0,
                size=5,
                marker=marker,
                dodge=True,
                legend=False,
            )
    elif plot_type == 'team':
        # --- Plotting ---
        # 1. Create the box plot to show the distribution of lap times for each team.
        #    This gives a good overview of each team's pace consistency.
        # Dry Tires
        sns.boxplot(
            data=driver_laps,
            x="Team",
            y="LapTime(s)",
            hue="Compound",
            order=team_order,
            palette=tire_palette,
            hue_order=used_compounds,
            fill=False,
            showfliers=False,
            legend=False,
            gap=0.1,
        )

        # 2. Overlay a swarm plot to show each individual valid lap.
        #    Each point is colored by the tyre compound used for that lap, providing
        #    deeper insight into the pace on different compounds.
        for session_no in range(session_idx):
            session_type = session_list[session_no].session_info['Type']
            marker = session_type_marker[session_type]
            tire_palette_adj = { key: to_rgba(val, alpha=(session_no + 1.)/session_idx) for key, val in tire_palette.items() }
            sns.swarmplot(
                data=driver_laps[ driver_laps['Session_Number'] == session_no ],
                x="Team",
                y="LapTime(s)",
                order=team_order,
                hue="Compound",
                palette=tire_palette_adj,
                hue_order=used_compounds,
                linewidth=0,
                size=5,
                marker=marker,
                dodge=True,
                legend=False,
            )
    return fig

msgStyle = {
        "flagColor": {
            "GREEN": 5763719,
            "CLEAR": 5763719,
            "YELLOW": 16776960,
            "DOUBLE YELLOW": 16776960,
            "CHEQUERED": 16777215,
            "BLUE": 3447003,
            "RED": 15548997,
            "BLACK AND WHITE": 16777215,
            "BLACK AND ORANGE": 15105570,
            "BLACK": 2303786,
        },
        "flagSymbol": {"CHEQUERED": ":checkered_flag:", "BLACK": ":flag_black:"},
        "modeColor": {"SAFETY CAR": 15844367, "VIRTUAL SAFETY CAR": 15844367},
        "compoundColor": {
            "SOFT": 15548997,  # RED
            "MEDIUM": 16776960,  # YELLOW
            "HARD": 16777215,  # WHITE
            "INTERMEDIATE": 2067276,  # GREEN
            "WET": 2123412,  # BLUE
        },
        "compoundRGB": {
            "WET": "#0067ad",
            "INTERMEDIATE": "#43b02a",
            "SOFT": "#da291c",
            "MEDIUM": "#ffd12e",
            "HARD": "#f0f0ec",
            # "UNKNOWN": "#00ffff",
            # "TEST-UNKNOWN": "#434649",
        },
        "compoundSymbol": {},
        "raceDirector": "Race Director",
    }


### Driver Pace

In [ ]:
# all completed sessions
fig = pace_plot('driver', session_idx['year'], session_idx['event'], session_idx['session'], driverList)
fig.show()

### Team Pace

In [ ]:
fig = pace_plot('team', session_idx['year'], session_idx['event'], session_idx['session'], driverList)
fig.show()